In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.rename({'default.payment.next.month': 'Default'},axis ='columns',inplace = True)

In [ ]:
df.head()

In [ ]:
df.drop('ID',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
#missing data

In [ ]:
df.dtypes

In [ ]:
df['SEX'].unique()

In [ ]:
df['EDUCATION'].unique()

In [ ]:
df['MARRIAGE'].unique()

In [ ]:
df['AGE'].unique()

In [ ]:
df['PAY_0'].unique()

In [ ]:
df['PAY_2'].unique()

In [ ]:
df['PAY_3'].unique()

In [ ]:
df['PAY_4'].unique()

In [ ]:
df['PAY_5'].unique()

In [ ]:
df['PAY_6'].unique()

In [ ]:
df['BILL_AMT1'].unique()

In [ ]:
df['BILL_AMT2'].unique()

In [ ]:
df['BILL_AMT3'].unique()

In [ ]:
df['BILL_AMT4'].unique()

In [ ]:
df['BILL_AMT5'].unique()

In [ ]:
df['BILL_AMT6'].unique()

In [ ]:
df['Default'].unique()

In [ ]:
cols = ['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

In [ ]:
float_col = df.select_dtypes(include=['float64'])

In [ ]:
for col in float_col.columns.values:
    df[col] = df[col].astype('int64')

In [ ]:
df.dtypes

In [ ]:
len(df.loc[(df['EDUCATION'] == 0) | (df['MARRIAGE'] == 0)])

In [ ]:
len(df.loc[(df['EDUCATION']==0) | (df['MARRIAGE'] == 0)])

In [ ]:
len(df)

In [ ]:
df_no_missing = df.loc[(df['EDUCATION'] != 0) & (df['MARRIAGE'] != 0)]

In [ ]:
len(df_no_missing)

In [ ]:
df_no_missing['EDUCATION'].unique()

In [ ]:
df_no_missing['MARRIAGE'].unique()

In [ ]:
df_no_default = df_no_missing[df_no_missing['Default']== 0]
df_default = df_no_missing[df_no_missing['Default'] == 1]

In [ ]:
from sklearn.utils import resample
df_no_default_downsample = resample(df_no_default,replace = False,n_samples=1000,random_state=42)

In [ ]:
len(df_no_default_downsample)

In [ ]:
df_default_downsample = resample(df_default,replace = False,n_samples=1000,random_state=42)

In [ ]:
len(df_default_downsample)

In [ ]:
df_downsample = pd.concat([df_no_default_downsample,df_default_downsample])

In [ ]:
len(df_downsample)

In [ ]:
#formating the data
X = df_downsample.drop('Default',axis =1).copy()
X.head()

In [ ]:
y = df_downsample['Default'].copy()
y.head()

In [ ]:
df

In [ ]:
#formate : one hot coding 
X_encoded = pd.get_dummies(X,columns=['SEX','EDUCATION','MARRIAGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'])

In [ ]:
X_encoded

In [ ]:
#formate3 scalling and centering 
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_encoded,y,random_state = 42)
X_train_scaled = scale(X_train)
X_test_scaled = scale(X_test)

In [ ]:
from sklearn.svm import SVC
#Building SVM
clf_svm = SVC(random_state = 42)
clf_svm.fit(X_train_scaled,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf_svm,X_test_scaled,y_test,values_format ='d',display_labels = ["did not default","defaulted"])

In [ ]:
from sklearn.model_selection import GridSearchCV
#optimizing parameters with cross validationa and grid search
param_grid = [
    {'C': [0.5,1,10,100],
    'gamma':['scale',1,0.1,0.01,0.001,0.0001],
    'kernel':['rbf']},
]

optimal_pramas = GridSearchCV(
                SVC(),
                param_grid,
                cv = 5,
                scoring = 'accuracy',
                verbose = 0)

optimal_pramas.fit(X_train_scaled,y_train)
print(optimal_pramas.best_params_)

In [ ]:
from sklearn.svm import SVC
#Building SVM
clf_svm = SVC(random_state = 42,C=100,gamma =0.001)
clf_svm.fit(X_train_scaled,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf_svm,X_test_scaled,y_test,values_format ='d',display_labels = ["did not default","defaulted"])

In [ ]:
from sklearn.decomposition import PCA
# pca
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)
per_var = np.round(pca.explained_variance_ratio_* 100,decimals =1)
labels = [str(x) for x in range(1,len(per_var)+1)]
plt.bar(x = range(1,len(per_var)+1),height= per_var)
plt.tick_params(
        axis= 'x',
        which = 'both',
        bottom = False,
        top = False,
        labelbottom= False)
plt.ylabel('percentage of variance explained')
plt.xlabel('principal component')
plt.title('scree plot')
plt.show